<a href="https://colab.research.google.com/github/asimds/gupshup/blob/main/GUPSHUP_%2Bemail%2Bpayment_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Compilation of work for messages dataset shared in three csv files named as A,B & C*** 

In [1]:
#importing libraries

import pandas as pd
import numpy as np
import seaborn as sns

#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###***Loading dataset from 3 files (around 6 GB) & merged into one file as a pickle*** 

**columns like '2000198658', 'MASK', 'TRANSACTION ID', 'RETRY STATUS', 'PRINCIPAL ENTITYID' already been dropped before merging 3 csv file.**

In [2]:
df = pd.read_pickle('/content/drive/MyDrive/26Dec21/df_abc_merged')
df.shape
#(22754203, 11) ~ 2GB

(22754203, 11)

***Using library to reduce dataset size***

In [9]:
#!pip install dfreduce
#from dfreduce.core import DFReduce
#DFReduce(df, inplace=True).reduce()
#OUTPUT-
#1it [00:00, 276.60it/s]
#1it [00:00, 665.23it/s]
#100%|██████████| 11/11 [01:38<00:00,  8.92s/it]
#Memory reduced from 16899.85 MB to 2731.08 MB
#Reduced by 83.84% !!
#save that reduced file as a pickle
#df.to_pickle('/content/drive/MyDrive/26Dec21/df_abc_reduced')

In [10]:
#import pandas as pd
#df=pd.read_pickle('/content/drive/MyDrive/26Dec21/df_abc_reduced')

In [16]:
df.head(2)

,MOBILE_NUMBER,MESSAGE_ID,MESSAGE_TEXT,TIMESTAMP,STATUS,CAUSE,LATENCY(MINS),SMSTYPE,NUMBER_MESSAGES,DELIVERY_CODE,DLT_TEMPLATEID
0,919958839124,86937918634223399,<#> Your GalaxyCard OTP is 1234. Do not share ...,2021-06-05 16:03:31,Delivered,Delivered,0.0,TEXT,1,000,1107161518935073113
1,916201108256,273883812086644956,<#> Your GalaxyCard OTP is 480544. Do not shar...,2021-06-05 20:41:58,Delivered,Delivered,0.0,TEXT,1,000,1107161518935073113


In [17]:
df.columns

Index(['MOBILE_NUMBER', 'MESSAGE_ID', 'MESSAGE_TEXT', 'TIMESTAMP', 'STATUS',
       'CAUSE', 'LATENCY(MINS)', 'SMSTYPE', 'NUMBER_MESSAGES', 'DELIVERY_CODE',
       'DLT_TEMPLATEID'],
      dtype='object')

***We can use TIMESTAMP & LATENCY to create a new column as MESSAGE DELIVERED to analyse attributes related to time and message delivered such as part of the day, week, month and so on but the objective of problem statement is analyse user behaviour with respect to messages sent and their type.***

In [3]:
#dropping unnecessary columns
unnecessary = ['MESSAGE_ID','NUMBER_MESSAGES', 'DELIVERY_CODE','TIMESTAMP','LATENCY(MINS)','MESSAGE_TEXT']

df.drop(unnecessary,axis=1,inplace=True)

In [4]:
#creating function for adding message type with respect to TEMPLATE
def msg_type(row):
  if row['DLT_TEMPLATEID'] == '1107161821627156186':
    return 'REFERENCE'
  elif row['DLT_TEMPLATEID'] == '1107161785922013990':
    return 'REFERENCE'
  elif row['DLT_TEMPLATEID'] == '1107163366925685914':
    return 'REFERENCE'
  elif row['DLT_TEMPLATEID'] == '1107161518935073113':
    return 'OTP'
  elif row['DLT_TEMPLATEID'] == '1107161175141807843':
    return 'OVERDUE'  
  elif row['DLT_TEMPLATEID'] == '1107161959036451136':
    return 'OVERDUE'    
  elif row['DLT_TEMPLATEID'] == '1107162756669414535':
    return 'REFERENCE_REGIONAL'    
  elif row['DLT_TEMPLATEID'] == '1107162756643364843':
    return 'PAYMENT_LINK'    
  elif row['DLT_TEMPLATEID'] == '1107162081293342527':
    return 'CIBIL_OVERDUE'    
  elif row['DLT_TEMPLATEID'] == '1107162072741862430':
    return 'OVERDUE REGIONAL'    
  elif row['DLT_TEMPLATEID'] == '1107161513051556632':
    return 'REMINDER'    
  elif row['DLT_TEMPLATEID'] == '1107162764721508748':
    return 'OVERDUE_SETTLEMENT'
  elif row['DLT_TEMPLATEID'] == '1107161546691488861':
    return 'REFERENCE'
  elif row['DLT_TEMPLATEID'] == '1107161175183783550':
    return 'CIBIL_OVERDUE'            
  elif row['DLT_TEMPLATEID'] == '1107161546722606834':
    return 'ISSUE_RESOLVED'
  elif row['DLT_TEMPLATEID'] == '1107161182158375609':
   return 'SETTLEMENT'
  elif row['DLT_TEMPLATEID'] == '1107161520621426151':
    return 'REFERENCE'    
  else:
    return 'Undefined'

In [5]:
#adding new column to identify what type of messages sent w.r.t TEMPLATE_ID
df['MESSAGE_TYPE'] = df.apply(msg_type, axis=1)

In [6]:
#checking type of messages with their value counts
df.MESSAGE_TYPE.value_counts()

REFERENCE             21633676
OVERDUE                 461856
OTP                     374402
REFERENCE_REGIONAL       92348
PAYMENT_LINK             90844
CIBIL_OVERDUE            58638
OVERDUE REGIONAL         23443
REMINDER                  6613
OVERDUE_SETTLEMENT        6510
Undefined                 2814
ISSUE_RESOLVED            2444
SETTLEMENT                 615
Name: MESSAGE_TYPE, dtype: int64

In [7]:
#checking null values
df.isna().sum()

MOBILE_NUMBER        0
STATUS               0
CAUSE                0
SMSTYPE              0
DLT_TEMPLATEID    2803
MESSAGE_TYPE         0
dtype: int64

In [8]:
#discarding those messages type where we have undefined message type (cases where message text is itself null) 
df=df.loc[df['MESSAGE_TYPE'] != 'Undefined']

In [9]:
df.shape

(22751389, 6)

In [10]:
df.MESSAGE_TYPE.value_counts()

REFERENCE             21633676
OVERDUE                 461856
OTP                     374402
REFERENCE_REGIONAL       92348
PAYMENT_LINK             90844
CIBIL_OVERDUE            58638
OVERDUE REGIONAL         23443
REMINDER                  6613
OVERDUE_SETTLEMENT        6510
ISSUE_RESOLVED            2444
SETTLEMENT                 615
Name: MESSAGE_TYPE, dtype: int64

In [11]:
df.SMSTYPE.value_counts()

TEXT            22167129
FLASH             468469
UNICODE_TEXT      115791
Name: SMSTYPE, dtype: int64

In [12]:
df.shape

(22751389, 6)

In [13]:
#dropping different type of messages sent encoding to focus on text messages only
df = df.loc[df['SMSTYPE'] != 'UNICODE_TEXT']
df = df.loc[df['SMSTYPE'] != 'FLASH']

In [14]:
#dropping columns since we now have only one message type as text and mapped with what type of msgs are there w.r.t template shared by SABA
df.drop(['SMSTYPE','DLT_TEMPLATEID'], axis=1, inplace=True)

In [15]:
df.shape

(22167129, 4)

In [16]:
df.head()

,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE
0,919958839124,Delivered,Delivered,OTP
1,916201108256,Delivered,Delivered,OTP
2,916204839638,Delivered,Delivered,OTP
3,919911039015,Delivered,Delivered,OTP
4,916201108256,Delivered,Delivered,OTP


In [17]:
#removing country code from all mobile numbers
import numpy as np
df['MOBILE_NUMBER'] = df['MOBILE_NUMBER'].astype(str).str[2:12].astype(np.int64)

In [18]:
df.head()

,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE
0,9958839124,Delivered,Delivered,OTP
1,6201108256,Delivered,Delivered,OTP
2,6204839638,Delivered,Delivered,OTP
3,9911039015,Delivered,Delivered,OTP
4,6201108256,Delivered,Delivered,OTP


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22167129 entries, 0 to 6754377
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   MOBILE_NUMBER  int64 
 1   STATUS         object
 2   CAUSE          object
 3   MESSAGE_TYPE   object
dtypes: int64(1), object(3)
memory usage: 845.6+ MB


In [20]:
df.columns

Index(['MOBILE_NUMBER', 'STATUS', 'CAUSE', 'MESSAGE_TYPE'], dtype='object')

In [21]:
#saving dataset
df.to_pickle('/content/drive/MyDrive/26Dec21/df_a+b+c')

In [22]:
#saving dataset
df.to_csv('/content/drive/MyDrive/26Dec21/df_a+b+c.csv')

#***To save crash start reading file from this check point***

In [67]:
#reading pickle file to save preprocessing of the above steps
import pandas as pd


In [68]:
df = pd.read_pickle('/content/drive/MyDrive/26Dec21/df_a+b+c')

###***Loading dataset from contacts to assign UserID to messages sent***

***We'll be using data from Contacts and Application to first assign appID with cooresponding contact and then from that appID we'll assign UserID.***

***Query for selecting dataset from Contacts***-
SELECT application_id, contact AS MOBILE_NUMBER, type
FROM contacts
where type = 'phone'


In [69]:
df1= pd.read_csv('/content/drive/MyDrive/26Dec21/contactscompile.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
df1.head()

,application_id,mobile_number,type
0,429835,8800582915,phone
1,429835,8826291656,phone
2,429835,8826511422,phone
3,429835,8851693057,phone
4,429835,8860410611,phone


In [71]:
#unique appiId 
df1.application_id.nunique()

6931

In [72]:
#unique phone number in dataset
df1.mobile_number.nunique()

1031969

In [73]:
df1.drop('type', axis=1, inplace=True)

In [74]:
#rename columns to merge
df1.rename(columns={'mobile_number': 'MOBILE_NUMBER'}, inplace=True)

In [75]:
#merging messages log & contact dataset
df_msg_plus_contact= pd.merge(df, df1, on='MOBILE_NUMBER')

In [76]:
df_msg_plus_contact.head(3)

,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE,application_id
0,7008928813,Delivered,Delivered,OTP,433977
1,7008928813,Delivered,Delivered,OTP,433977
2,7008928813,Delivered,Delivered,OTP,433977


In [77]:
df_msg_plus_contact.shape

(9389747, 5)

In [78]:
#unique user application id in merged dataset
df_msg_plus_contact.application_id.nunique()

1694

***Query for selecting dataset from application***- select id as applications_id, user_id  
from applications

In [79]:
#reading application id
df3=pd.read_csv('/content/drive/MyDrive/user_appid.csv')

In [80]:
df3.head()

,application_id,user_id
0,576495,636562
1,267172,331820
2,244165,307649
3,513012,580355
4,185235,240951


In [81]:
#unique application id present in dataset
df3.application_id.nunique()

610911

In [82]:
#merging prebius dataset with app id to get user id 
df_msg_plus_contact_plus_app= pd.merge(df_msg_plus_contact, df3, on='application_id')

In [83]:
df_msg_plus_contact_plus_app.head()

,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE,application_id,user_id
0,7008928813,Delivered,Delivered,OTP,433977,507722
1,7008928813,Delivered,Delivered,OTP,433977,507722
2,7008928813,Delivered,Delivered,OTP,433977,507722
3,7008928813,Delivered,Delivered,OTP,433977,507722
4,7008928813,Delivered,Delivered,OTP,433977,507722


In [84]:
df_msg_plus_contact_plus_app.drop('application_id', axis=1, inplace=True)

In [85]:
df_msg_plus_contact_plus_app.shape

(9389747, 5)

In [86]:
df_msg_plus_contact_plus_app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9389747 entries, 0 to 9389746
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   MOBILE_NUMBER  object
 1   STATUS         object
 2   CAUSE          object
 3   MESSAGE_TYPE   object
 4   user_id        int64 
dtypes: int64(1), object(4)
memory usage: 429.8+ MB


In [87]:
#swapping columns in dataframe
uid = df_msg_plus_contact_plus_app['user_id']
df_msg_plus_contact_plus_app.drop(labels=['user_id'], axis=1,inplace = True)
df_msg_plus_contact_plus_app.insert(0, 'user_id', uid)

In [88]:
df_msg_plus_contact_plus_app.head(3)

,user_id,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE
0,507722,7008928813,Delivered,Delivered,OTP
1,507722,7008928813,Delivered,Delivered,OTP
2,507722,7008928813,Delivered,Delivered,OTP


In [89]:
#converting object columns as string to use them in a pivot 
df_msg_plus_contact_plus_app.CAUSE = df_msg_plus_contact_plus_app.CAUSE.astype(str)
df_msg_plus_contact_plus_app.STATUS = df_msg_plus_contact_plus_app.STATUS.astype(str)

In [90]:
#creating pivot table
out = pd.pivot_table(data=df_msg_plus_contact_plus_app, 
                        index=['user_id',], 
                        columns=['MESSAGE_TYPE','STATUS'], 
                        values='CAUSE',
                        aggfunc='count').reset_index()


In [91]:
#flatenning hierarchical index
out.columns = [' '.join(col).strip() for col in out.columns.values]

In [92]:
out.head()

,user_id,CIBIL_OVERDUE Delivered,CIBIL_OVERDUE Operator Failed,ISSUE_RESOLVED Delivered,ISSUE_RESOLVED Operator Failed,OTP Delivered,OTP Not Sent to Operator,OTP Operator Failed,OTP Report Pending,OVERDUE_SETTLEMENT Delivered,OVERDUE_SETTLEMENT Operator Failed,PAYMENT_LINK Delivered,PAYMENT_LINK Operator Failed,REFERENCE Delivered,REFERENCE Not Sent to Operator,REFERENCE Operator Failed,REFERENCE Report Pending,SETTLEMENT Delivered,SETTLEMENT Operator Failed
0,27923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN
1,33399,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,1.0,NaN,NaN,NaN
3,36962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN
4,42867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


**A blank cell(NaN) indicates that there is no record/activity for that row and column label.**

In [93]:
#rermoving space from columns
out.columns = out.columns.str.replace(' ', '_')

In [94]:
out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1694 entries, 0 to 1693
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   user_id                             1694 non-null   int64  
 1   CIBIL_OVERDUE_Delivered             69 non-null     float64
 2   CIBIL_OVERDUE_Operator_Failed       30 non-null     float64
 3   ISSUE_RESOLVED_Delivered            46 non-null     float64
 4   ISSUE_RESOLVED_Operator_Failed      385 non-null    float64
 5   OTP_Delivered                       664 non-null    float64
 6   OTP_Not_Sent_to_Operator            8 non-null      float64
 7   OTP_Operator_Failed                 32 non-null     float64
 8   OTP_Report_Pending                  4 non-null      float64
 9   OVERDUE_SETTLEMENT_Delivered        22 non-null     float64
 10  OVERDUE_SETTLEMENT_Operator_Failed  10 non-null     float64
 11  PAYMENT_LINK_Delivered              90 non-

In [95]:
#filling null values
out.fillna(0,inplace=True)

In [96]:
out.columns

Index(['user_id', 'CIBIL_OVERDUE_Delivered', 'CIBIL_OVERDUE_Operator_Failed',
       'ISSUE_RESOLVED_Delivered', 'ISSUE_RESOLVED_Operator_Failed',
       'OTP_Delivered', 'OTP_Not_Sent_to_Operator', 'OTP_Operator_Failed',
       'OTP_Report_Pending', 'OVERDUE_SETTLEMENT_Delivered',
       'OVERDUE_SETTLEMENT_Operator_Failed', 'PAYMENT_LINK_Delivered',
       'PAYMENT_LINK_Operator_Failed', 'REFERENCE_Delivered',
       'REFERENCE_Not_Sent_to_Operator', 'REFERENCE_Operator_Failed',
       'REFERENCE_Report_Pending', 'SETTLEMENT_Delivered',
       'SETTLEMENT_Operator_Failed'],
      dtype='object')

In [97]:
#statistical description of dataset
out.describe()

,user_id,CIBIL_OVERDUE_Delivered,CIBIL_OVERDUE_Operator_Failed,ISSUE_RESOLVED_Delivered,ISSUE_RESOLVED_Operator_Failed,OTP_Delivered,OTP_Not_Sent_to_Operator,OTP_Operator_Failed,OTP_Report_Pending,OVERDUE_SETTLEMENT_Delivered,OVERDUE_SETTLEMENT_Operator_Failed,PAYMENT_LINK_Delivered,PAYMENT_LINK_Operator_Failed,REFERENCE_Delivered,REFERENCE_Not_Sent_to_Operator,REFERENCE_Operator_Failed,REFERENCE_Report_Pending,SETTLEMENT_Delivered,SETTLEMENT_Operator_Failed
count,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000,1694.000000
mean,437304.695986,0.336482,0.094451,0.027745,0.475797,1.912633,0.004723,0.042503,0.002952,0.028926,0.011806,0.531287,0.122196,575.409091,0.906730,4946.768595,16.265643,0.001771,0.000590
std,132364.533912,2.257782,1.158672,0.167847,1.094611,4.838384,0.068579,0.511498,0.064234,0.293242,0.178204,3.172333,1.432825,1285.959255,13.167091,8540.808084,28.087876,0.042058,0.024296
min,27923.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,358695.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,503263.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.500000,0.000000,146.000000,1.000000,0.000000,0.000000
75%,508950.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,299.000000,0.000000,6479.000000,18.000000,0.000000,0.000000
max,700279.000000,52.000000,30.000000,2.000000,11.000000,77.000000,1.000000,17.000000,2.000000,6.000000,4.000000,63.000000,36.000000,11144.000000,192.000000,45652.000000,154.000000,1.000000,1.000000


In [98]:
out.tail()

,user_id,CIBIL_OVERDUE_Delivered,CIBIL_OVERDUE_Operator_Failed,ISSUE_RESOLVED_Delivered,ISSUE_RESOLVED_Operator_Failed,OTP_Delivered,OTP_Not_Sent_to_Operator,OTP_Operator_Failed,OTP_Report_Pending,OVERDUE_SETTLEMENT_Delivered,OVERDUE_SETTLEMENT_Operator_Failed,PAYMENT_LINK_Delivered,PAYMENT_LINK_Operator_Failed,REFERENCE_Delivered,REFERENCE_Not_Sent_to_Operator,REFERENCE_Operator_Failed,REFERENCE_Report_Pending,SETTLEMENT_Delivered,SETTLEMENT_Operator_Failed
1689,700259,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,0.0,17891.0,59.0,0.0,0.0
1690,700261,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,332.0,0.0,14.0,0.0,0.0,0.0
1691,700266,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,652.0,0.0,21666.0,69.0,0.0,0.0
1692,700277,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,9.0,0.0,0.0,0.0
1693,700279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0


In [99]:
#saving pickle and csv files
out.to_pickle('/content/drive/MyDrive/26Dec21/gupshup_message_summary')

In [100]:
out.to_csv('/content/drive/MyDrive/26Dec21/gupshup_message_summary.csv')

###***Till now the work done is based on the CSV file shared in three part and analysing with respect to user to quantify the type of messages sent. This file can be now be merged with each users email and payment traits(feature)***

In [33]:
#reading gupshup messages log
import pandas as pd
out= pd.read_pickle('/content/drive/MyDrive/26Dec21/gupshup_message_summary')

In [34]:
#reading email logs
df_email=pd.read_csv('/content/drive/MyDrive/processed_email_logs.csv')

In [35]:
#renaming column
out.rename(columns={'user_id': 'User_ID'}, inplace=True)

In [36]:
#merging prebius dataset with app id to get user id 
df_gupshup_email= pd.merge(df_email, out, on='User_ID')

In [37]:
#shape of comnbined dataframe
df_gupshup_email.shape

(990, 106)

***So we now have email and messages data of around 990 people with 106 features.*** 

In [38]:
#checking head of dataframe
df_gupshup_email.head()

,Unnamed: 0,User_ID,is_overdue,days_in_overdue,api_typemails,campaign_typemails,email_typemails,api_Latest_Reopen_Time_Mean,campaign_Latest_Reopen_Time_Mean,email_Latest_Reopen_Time_Mean,api_Latest_Reopen_Time_Median,campaign_Latest_Reopen_Time_Median,email_Latest_Reopen_Time_Median,api_Latest_Reopen_Time_Min,campaign_Latest_Reopen_Time_Min,email_Latest_Reopen_Time_Min,api_Latest_Reopen_Time_Max,campaign_Latest_Reopen_Time_Max,email_Latest_Reopen_Time_Max,api_Oldest_Reopen_Time_Mean,campaign_Oldest_Reopen_Time_Mean,email_Oldest_Reopen_Time_Mean,api_Oldest_Reopen_Time_Median,campaign_Oldest_Reopen_Time_Median,email_Oldest_Reopen_Time_Median,api_Oldest_Reopen_Time_Min,campaign_Oldest_Reopen_Time_Min,email_Oldest_Reopen_Time_Min,api_Oldest_Reopen_Time_Max,campaign_Oldest_Reopen_Time_Max,email_Oldest_Reopen_Time_Max,api_Average_Reopen_Time,campaign_Average_Reopen_Time,email_Average_Reopen_Time,api_Median_Reopen_Time,campaign_Median_Reopen_Time,email_Median_Reopen_Time,api_First_Read_Delay_Mean,campaign_First_Read_Delay_Mean,email_First_Read_Delay_Mean,...,email_is_failed,api_viewed_in_browser,campaign_viewed_in_browser,email_viewed_in_browser,api_retry_count_Mean,campaign_retry_count_Mean,email_retry_count_Mean,api_retry_count_Median,campaign_retry_count_Median,email_retry_count_Median,api_retry_count_Sum,campaign_retry_count_Sum,email_retry_count_Sum,api_open_count_Mean,campaign_open_count_Mean,email_open_count_Mean,api_open_count_Median,campaign_open_count_Median,email_open_count_Median,api_open_count_Sum,campaign_open_count_Sum,email_open_count_Sum,CIBIL_OVERDUE_Delivered,CIBIL_OVERDUE_Operator_Failed,ISSUE_RESOLVED_Delivered,ISSUE_RESOLVED_Operator_Failed,OTP_Delivered,OTP_Not_Sent_to_Operator,OTP_Operator_Failed,OTP_Report_Pending,OVERDUE_SETTLEMENT_Delivered,OVERDUE_SETTLEMENT_Operator_Failed,PAYMENT_LINK_Delivered,PAYMENT_LINK_Operator_Failed,REFERENCE_Delivered,REFERENCE_Not_Sent_to_Operator,REFERENCE_Operator_Failed,REFERENCE_Report_Pending,SETTLEMENT_Delivered,SETTLEMENT_Operator_Failed
0,4166,286969.0,0.0,182.0,16.0,91.0,12.0,2335584.0,73477.0,79.0,145976.000000,818.516484,7.750000,2.0,3.0,0.0,0.0,0.0,0.0,2335584.0,73477.0,79.0,146787.312500,2316.494505,9.666667,15.5,10.0,0.0,0.0,0.0,0.0,146157.485417,1357.368498,8.708333,7.5,8.0,0.0,143231.0,328949.0,380027.0,...,12.0,0.0,0.0,0.0,2.687500,2.252747,1.666667,2.5,2.0,1.0,43.0,205.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,8.0,0.0,0.0,0.0
1,4822,295954.0,0.0,236.0,56.0,97.0,108.0,89923.0,107.0,9558798.0,3849.214286,2.175258,88508.296296,0.0,0.0,0.0,0.0,0.0,0.0,89923.0,77373.0,9558798.0,5092.089286,1595.319588,88508.296296,0.0,0.0,0.0,0.0,0.0,0.0,4388.110119,751.517182,88508.296296,0.0,0.0,0.0,3373546.0,4549923.0,9507003.0,...,108.0,0.0,0.0,0.0,1.250000,1.051546,1.250000,1.0,1.0,1.0,70.0,102.0,135.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,30.0,0.0,0.0,0.0
2,5083,270143.0,0.0,215.0,19.0,179.0,60.0,667.0,16105.0,61624.0,41.263158,1441.888268,1697.616667,0.0,0.0,0.0,0.0,0.0,0.0,1785.0,16105.0,61624.0,407.421053,2187.983240,2030.900000,0.0,0.0,0.0,0.0,0.0,0.0,180.795739,1777.002487,1863.958333,0.0,0.0,0.0,13563.0,98382.0,1633277.0,...,60.0,0.0,0.0,0.0,2.578947,1.759777,1.333333,1.0,1.0,1.0,49.0,315.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0
3,7083,367238.0,0.0,137.0,45.0,4.0,0.0,4920.0,0.0,0.0,109.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4920.0,0.0,0.0,109.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,109.333333,0.000000,0.000000,0.0,0.0,0.0,73487.0,9212.0,0.0,...,0.0,0.0,0.0,0.0,1.022222,1.000000,0.000000,1.0,1.0,0.0,46.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,1.0,0.0,0.0,0.0
4,7769,358570.0,0.0,147.0,70.0,389.0,10.0,2283681.0,1465409.0,19.0,32628.185714,9556.640103,2.500000,0.0,0.0,0.0,0.0,0.0,0.0,2283681.0,1465409.0,19.0,701

###***Working on payments data. Query is mentioned below***

In [39]:
#SELECT a.id, a.user_id,payments_new.amount,payments_new.entity_type,payments_new.settled_at,payments_new.due_date,a.product FROM payments_new
#        LEFT JOIN payments_new r ON r.entity_type = 'payment' AND r.entity_id::uuid = payments_new.uuid
#        join applications a on payments_new.user_id = a.user_id
#        WHERE r.uuid IS NULL
#        AND (payments_new.entity_type NOT IN ('bill', 'credit', 'payment') OR (payments_new.entity_type = 'bill' AND payments_new.processed = TRUE))
#        and payments_new.entity_type in ('buy_gold','emi','interest','late_fee','utility','shopping','charge','giftcard','qr','convenience_fee','dnd_fee','emi_processing_fee')
#        and payments_new.emi_duration = 0

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import pandas as pd

In [42]:
#reading payments csv file
df4=pd.read_csv('/content/drive/MyDrive/26Dec21/payments.csv')

In [43]:
df4.head()

,id,user_id,amount,entity_type,settled_at,due_date,product
0,93713,129758,-20000,utility,2021-11-30T00:00:00+05:30,2021-12-06T05:29:59+05:30,automated
1,93713,129758,-80000,giftcard,2022-01-01T00:00:00+05:30,2022-01-05T05:29:59+05:30,automated
2,93713,129758,-20000,giftcard,2022-01-01T00:00:00+05:30,2022-01-05T05:29:59+05:30,automated
3,93713,129758,-100000,giftcard,NaN,2022-02-05T05:29:59+05:30,automated
4,581908,641874,-1000,qr,2022-01-11T00:00:00+05:30,2022-01-09T05:29:59+05:30,automated


In [44]:
df4.columns

Index(['id', 'user_id', 'amount', 'entity_type', 'settled_at', 'due_date',
       'product'],
      dtype='object')

In [45]:
df4['product'].value_counts()

automated       304653
secured_card     43052
Name: product, dtype: int64

In [46]:
#converting into datetime column
df4[['settled_at']] = df4[['settled_at']].apply(pd.to_datetime)
df4[['due_date']] = df4[['due_date']].apply(pd.to_datetime)

In [47]:
df4.drop(['id'],axis=1,inplace=True)

In [48]:
df4.head()

,user_id,amount,entity_type,settled_at,due_date,product
0,129758,-20000,utility,2021-11-30 00:00:00+05:30,2021-12-06 05:29:59+05:30,automated
1,129758,-80000,giftcard,2022-01-01 00:00:00+05:30,2022-01-05 05:29:59+05:30,automated
2,129758,-20000,giftcard,2022-01-01 00:00:00+05:30,2022-01-05 05:29:59+05:30,automated
3,129758,-100000,giftcard,NaT,2022-02-05 05:29:59+05:30,automated
4,641874,-1000,qr,2022-01-11 00:00:00+05:30,2022-01-09 05:29:59+05:30,automated


In [49]:
#finding DPD
df4['due_to_settled_date_diff'] = (df4['settled_at'] - df4['due_date']).dt.days

In [50]:
df4.head()

,user_id,amount,entity_type,settled_at,due_date,product,due_to_settled_date_diff
0,129758,-20000,utility,2021-11-30 00:00:00+05:30,2021-12-06 05:29:59+05:30,automated,-7.0
1,129758,-80000,giftcard,2022-01-01 00:00:00+05:30,2022-01-05 05:29:59+05:30,automated,-5.0
2,129758,-20000,giftcard,2022-01-01 00:00:00+05:30,2022-01-05 05:29:59+05:30,automated,-5.0
3,129758,-100000,giftcard,NaT,2022-02-05 05:29:59+05:30,automated,NaN
4,641874,-1000,qr,2022-01-11 00:00:00+05:30,2022-01-09 05:29:59+05:30,automated,1.0


***Use groupby and apply fillna with median. Then get NaN if some category has only NaN values, so use mean of all values of column for filling NaN***

In [51]:
df4.due_to_settled_date_diff = df4.groupby('user_id')['due_to_settled_date_diff'].apply(lambda x: x.fillna(x.median()))
df4.due_to_settled_date_diff = df4.due_to_settled_date_diff.fillna(df4.due_to_settled_date_diff.mean())

In [91]:
#rounding of to get the more rigid picture
df4['due_to_settled_date_diff'] = round(df4['due_to_settled_date_diff'])

In [53]:
df4.isna().sum()

user_id                          0
amount                           0
entity_type                      0
settled_at                  107601
due_date                         0
product                        727
due_to_settled_date_diff         0
dtype: int64

In [54]:
df4.head()

,user_id,amount,entity_type,settled_at,due_date,product,due_to_settled_date_diff
0,129758,-20000,utility,2021-11-30 00:00:00+05:30,2021-12-06 05:29:59+05:30,automated,-7.0
1,129758,-80000,giftcard,2022-01-01 00:00:00+05:30,2022-01-05 05:29:59+05:30,automated,-5.0
2,129758,-20000,giftcard,2022-01-01 00:00:00+05:30,2022-01-05 05:29:59+05:30,automated,-5.0
3,129758,-100000,giftcard,NaT,2022-02-05 05:29:59+05:30,automated,-5.0
4,641874,-1000,qr,2022-01-11 00:00:00+05:30,2022-01-09 05:29:59+05:30,automated,1.0


In [55]:
df4.drop(['settled_at','due_date'],axis=1,inplace=True)

In [56]:
#filling remaining null values with the mode
df4 = df4.fillna(df4.mode().iloc[0])

In [57]:
df4.isna().sum()

user_id                     0
amount                      0
entity_type                 0
product                     0
due_to_settled_date_diff    0
dtype: int64

In [58]:
df4.tail()

,user_id,amount,entity_type,product,due_to_settled_date_diff
348427,228214,-29000,emi,automated,182.0
348428,228214,-3413,late_fee,automated,182.0
348429,228214,-29000,emi,automated,-7.0
348430,228214,-4720,emi_processing_fee,automated,-7.0
348431,228214,-3422,late_fee,automated,151.0


In [59]:
#creating amount summary and other attributes summary with respect to expenditure of user selected in query
import numpy as np

amt_smm = df4.groupby('user_id')['amount','due_to_settled_date_diff'].agg([np.sum,np.mean, np.max, np.min]).reset_index()    
amt_smm.columns = ['user_id', 'amount_total', 'amount_mean', 'amount_max', 'amount_min','due_to_settled_date_diff_total','due_to_settled_date_diff_mean','due_to_settled_date_diff_max','due_to_settled_date_diff_min' ]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
amt_smm.head()

,user_id,amount_total,amount_mean,amount_max,amount_min,due_to_settled_date_diff_total,due_to_settled_date_diff_mean,due_to_settled_date_diff_max,due_to_settled_date_diff_min
0,1,-4400016,-112820.923077,-1,-300000,-11510.0,-295.128205,-4.0,-1057.0
1,7,-55000,-55000.000000,-55000,-55000,4.0,4.000000,4.0,4.0
2,25428,-127800,-14200.000000,-1000,-39900,-18.0,-2.000000,-2.0,-2.0
3,25430,-70000,-70000.000000,-70000,-70000,870.0,870.000000,870.0,870.0
4,25431,-107809,-8984.083333,0,-50000,4143.0,345.250000,966.0,-1.0


In [61]:
amt_smm.shape

(27641, 9)

In [62]:
#separating entity type to analyse each users spending behaviour
df_et = df4[['user_id', 'entity_type']]

In [63]:
df_et.head()

,user_id,entity_type
0,129758,utility
1,129758,giftcard
2,129758,giftcard
3,129758,giftcard
4,641874,qr


In [64]:
df_et['entity_type'].value_counts()

emi                   97914
qr                    55391
utility               55037
convenience_fee       54590
late_fee              29716
emi_processing_fee    26148
giftcard              14660
charge                 5975
buy_gold               5385
dnd_fee                3393
shopping                222
interest                  1
Name: entity_type, dtype: int64

In [65]:
#We use pandas's 'get_dummies()' method
df_et = pd.get_dummies(df_et)

In [66]:
# Use GroupBy() to compute the sum
df_en_feat = df_et.groupby('user_id').sum()
#resetting index of dataframe
df_en_feat.reset_index(inplace = True)
df_en_feat.head()

,user_id,entity_type_buy_gold,entity_type_charge,entity_type_convenience_fee,entity_type_dnd_fee,entity_type_emi,entity_type_emi_processing_fee,entity_type_giftcard,entity_type_interest,entity_type_late_fee,entity_type_qr,entity_type_shopping,entity_type_utility
0,1,1.0,0.0,1.0,0.0,0.0,0.0,25.0,0.0,3.0,0.0,0.0,9.0
1,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,25428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0
3,25430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,25431,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0


In [75]:
#creating summary with respect to entity type of each user
df_etype_smm = df_et.groupby('user_id')['entity_type_buy_gold', 'entity_type_late_fee', 'entity_type_emi_processing_fee',  'entity_type_qr', 'entity_type_giftcard', 'entity_type_utility', 'entity_type_shopping', 'entity_type_charge', 'entity_type_dnd_fee', 'entity_type_convenience_fee', 'entity_type_emi'].agg([np.sum,np.mean, np.max, np.min]).reset_index()   


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [76]:
#renaming columns
df_etype_smm.columns = ['user_id',  'entity_type_buy_gold_total','entity_type_buy_gold_mean','entity_type_buy_gold_max','entity_type_buy_gold_min',
                         'entity_type_late_fee_total','entity_type_late_fee_mean','entity_type_late_fee_max','entity_type_late_fee_min',
                         'entity_type_emi_processing_fee_total','entity_type_emi_processing_fee_mean','entity_type_emi_processing_fee_max','entity_type_emi_processing_fee_min',
                         'entity_type_qr_total','entity_type_qr_mean','entity_type_qr_max','entity_type_qr_min',
                         'entity_type_giftcard_total','entity_type_giftcard_mean','entity_type_giftcard_max' ,'entity_type_giftcard_min',
                         'entity_type_utility_total','entity_type_utility_mean','entity_type_utility_max','entity_type_utility_min',
                         'entity_type_shopping_total','entity_type_shopping_mean','entity_type_shopping_max','entity_type_shopping_min',
                         'entity_type_charge_total','entity_type_charge_mean','entity_type_charge_max','entity_type_charge_min',
                         'entity_type_dnd_fee_total','entity_type_dnd_fee_mean','entity_type_dnd_fee_max','entity_type_dnd_fee_min',
                         'entity_type_convenience_fee_total','entity_type_convenience_fee_mean','entity_type_convenience_fee_max','entity_type_convenience_fee_min',
                         'entity_type_emi_total','entity_type_emi_mean','entity_type_emi_max','entity_type_emi_min' ]


In [77]:
df_etype_smm.head(3)

,user_id,entity_type_buy_gold_total,entity_type_buy_gold_mean,entity_type_buy_gold_max,entity_type_buy_gold_min,entity_type_late_fee_total,entity_type_late_fee_mean,entity_type_late_fee_max,entity_type_late_fee_min,entity_type_emi_processing_fee_total,entity_type_emi_processing_fee_mean,entity_type_emi_processing_fee_max,entity_type_emi_processing_fee_min,entity_type_qr_total,entity_type_qr_mean,entity_type_qr_max,entity_type_qr_min,entity_type_giftcard_total,entity_type_giftcard_mean,entity_type_giftcard_max,entity_type_giftcard_min,entity_type_utility_total,entity_type_utility_mean,entity_type_utility_max,entity_type_utility_min,entity_type_shopping_total,entity_type_shopping_mean,entity_type_shopping_max,entity_type_shopping_min,entity_type_charge_total,entity_type_charge_mean,entity_type_charge_max,entity_type_charge_min,entity_type_dnd_fee_total,entity_type_dnd_fee_mean,entity_type_dnd_fee_max,entity_type_dnd_fee_min,entity_type_convenience_fee_total,entity_type_convenience_fee_mean,entity_type_convenience_fee_max,entity_type_convenience_fee_min,entity_type_emi_total,entity_type_emi_mean,entity_type_emi_max,entity_type_emi_min
0,1,1,0.025641,1,0,3,0.076923,1,0,0,0.0,0,0,0.0,0.0,0,0,25.0,0.641026,1,0,9,0.230769,1,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,1.0,0.025641,1,0,0,0.0,0,0
1,7,0,0.000000,0,0,0,0.000000,0,0,0,0.0,0,0,0.0,0.0,0,0,0.0,0.000000,0,0,1,1.000000,1,1,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0.000000,0,0,0,0.0,0,0
2,25428,0,0.000000,0,0,4,0.444444,1,0,0,0.0,0,0,0.0,0.0,0,0,0.0,0.000000,0,0,5,0.555556,1,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0.000000,0,0,0,0.0,0,0


In [78]:
#merging dataset of amount summary and entity type summary
df_w1= pd.merge(amt_smm,df_etype_smm, on='user_id')

In [79]:
df_w1.shape

(27641, 53)

***Now merging this dataframe with gupshup logs***

In [80]:
#renaming column
df_w1.rename(columns={'user_id': 'User_ID'}, inplace=True)

In [81]:
df_final= pd.merge(df_gupshup_email,df_w1, on='User_ID')

In [82]:
df_final.head(4)

,Unnamed: 0,User_ID,is_overdue,days_in_overdue,api_typemails,campaign_typemails,email_typemails,api_Latest_Reopen_Time_Mean,campaign_Latest_Reopen_Time_Mean,email_Latest_Reopen_Time_Mean,api_Latest_Reopen_Time_Median,campaign_Latest_Reopen_Time_Median,email_Latest_Reopen_Time_Median,api_Latest_Reopen_Time_Min,campaign_Latest_Reopen_Time_Min,email_Latest_Reopen_Time_Min,api_Latest_Reopen_Time_Max,campaign_Latest_Reopen_Time_Max,email_Latest_Reopen_Time_Max,api_Oldest_Reopen_Time_Mean,campaign_Oldest_Reopen_Time_Mean,email_Oldest_Reopen_Time_Mean,api_Oldest_Reopen_Time_Median,campaign_Oldest_Reopen_Time_Median,email_Oldest_Reopen_Time_Median,api_Oldest_Reopen_Time_Min,campaign_Oldest_Reopen_Time_Min,email_Oldest_Reopen_Time_Min,api_Oldest_Reopen_Time_Max,campaign_Oldest_Reopen_Time_Max,email_Oldest_Reopen_Time_Max,api_Average_Reopen_Time,campaign_Average_Reopen_Time,email_Average_Reopen_Time,api_Median_Reopen_Time,campaign_Median_Reopen_Time,email_Median_Reopen_Time,api_First_Read_Delay_Mean,campaign_First_Read_Delay_Mean,email_First_Read_Delay_Mean,...,entity_type_late_fee_total,entity_type_late_fee_mean,entity_type_late_fee_max,entity_type_late_fee_min,entity_type_emi_processing_fee_total,entity_type_emi_processing_fee_mean,entity_type_emi_processing_fee_max,entity_type_emi_processing_fee_min,entity_type_qr_total,entity_type_qr_mean,entity_type_qr_max,entity_type_qr_min,entity_type_giftcard_total,entity_type_giftcard_mean,entity_type_giftcard_max,entity_type_giftcard_min,entity_type_utility_total,entity_type_utility_mean,entity_type_utility_max,entity_type_utility_min,entity_type_shopping_total,entity_type_shopping_mean,entity_type_shopping_max,entity_type_shopping_min,entity_type_charge_total,entity_type_charge_mean,entity_type_charge_max,entity_type_charge_min,entity_type_dnd_fee_total,entity_type_dnd_fee_mean,entity_type_dnd_fee_max,entity_type_dnd_fee_min,entity_type_convenience_fee_total,entity_type_convenience_fee_mean,entity_type_convenience_fee_max,entity_type_convenience_fee_min,entity_type_emi_total,entity_type_emi_mean,entity_type_emi_max,entity_type_emi_min
0,4166,286969.0,0.0,182.0,16.0,91.0,12.0,2335584.0,73477.0,79.0,145976.000000,818.516484,7.750000,2.0,3.0,0.0,0.0,0.0,0.0,2335584.0,73477.0,79.0,146787.312500,2316.494505,9.666667,15.5,10.0,0.0,0.0,0.0,0.0,146157.485417,1357.368498,8.708333,7.5,8.0,0.0,143231.0,328949.0,380027.0,...,2,0.250000,1,0,1,0.125000,1,0,0.0,0.000,0,0,2.0,0.25,1,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0.000,0,0,3,0.375000,1,0
1,4822,295954.0,0.0,236.0,56.0,97.0,108.0,89923.0,107.0,9558798.0,3849.214286,2.175258,88508.296296,0.0,0.0,0.0,0.0,0.0,0.0,89923.0,77373.0,9558798.0,5092.089286,1595.319588,88508.296296,0.0,0.0,0.0,0.0,0.0,0.0,4388.110119,751.517182,88508.296296,0.0,0.0,0.0,3373546.0,4549923.0,9507003.0,...,3,0.300000,1,0,1,0.100000,1,0,0.0,0.000,0,0,0.0,0.00,0,0,3,0.3,1,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0.000,0,0,3,0.300000,1,0
2,5083,270143.0,0.0,215.0,19.0,179.0,60.0,667.0,16105.0,61624.0,41.263158,1441.888268,1697.616667,0.0,0.0,0.0,0.0,0.0,0.0,1785.0,16105.0,61624.0,407.421053,2187.983240,2030.900000,0.0,0.0,0.0,0.0,0.0,0.0,180.795739,1777.002487,1863.958333,0.0,0.0,0.0,13563.0,98382.0,1633277.0,...,2,0.250000,1,0,1,0.125000,1,0,1.0,0.125,1,0,0.0,0.00,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,1.0,0.125,1,0,3,0.375000,1,0
3,7083,367238.0,0.0,137.0,45.0,4.0,0.0,4920.0,0.0,0.0,109.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4920.0,0.0,0.0,109.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,109.333333,0.000000,0.000000,0.0,0.0,0.0,73487.0,9212.0,0.0,...,3,0.272727,1,0,2,0.181818,1,0,0.0,0.000,0,0,0.0,0.00,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0.000,0,0,6,0.545455,1,0


In [83]:
df_final.shape

(710, 158)

In [86]:
df_final.drop('Unnamed: 0',axis=1,inplace=True)

In [88]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 710 entries, 0 to 709
Columns: 157 entries, User_ID to entity_type_emi_min
dtypes: float64(124), int64(3), uint8(30)
memory usage: 730.8 KB


In [90]:
df_final.isnull().values.any()

False

In [87]:
#saving pickle and csv files
df_final.to_pickle('/content/drive/MyDrive/26Dec21/final_data_processed')
df_final.to_csv('/content/drive/MyDrive/26Dec21/final_data_processed.csv')

###***The work done above can now be used as an input to cluster customers and target them separately post further analysis and according to EDA. We can also create a seprate dataframe after assigning clusters to each user and merging them with payment to capture their behaviour and how we reach out to them via messages and other modes***